In [1]:
from mx.progress import init_with_progress
init_with_progress()
from mx.prelude import *

AttributeError: module 'mx' has no attribute 'init'

In [ ]:
from mx import datasets as mxd, tasks as mxt, models as mxm, embedding as mxe, layers as mxl
from mx.pipeline import Pipeline

bavecdo = Pipeline(
    name="bavecdo",
    desc="Decoder-only transformer for vector regression on BVH data",
    batch_size=32,
    n_steps=1000,
    model=mxm.DecoderOnlyTransformer(
        n_layers=4,
        n_heads=16,
        n_hidden=2048,
    ),
    embedding=mxe.TransformerAngleVectorEmbedding(
        n_embd=512,
        n_repeats=12,
    ),
    task=mxt.NextUnitVectorPrediction(
        chunk_size=32,
    ),
    dataset=mxd.BvhDataset(
        recluster=False,
        decimate=False,
    ),
)

In [ ]:
pipeline = bavecdo

In [ ]:
from mx.train import build_for_training
(
    model,
    loss_fn,
    data,
    vizr,
    train_loop,
) = build_for_training(pipeline)

/home/maxeonyx/.local/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [ ]:
u.random_run_name()
train_loop()


KeyboardInterrupt

